In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open('logistic_regression_model.pkl', 'rb') as f:
    logistic_regression_model = pickle.load(f)

In [3]:
with open('tuned_xgboost_model.pkl', 'rb') as f:
    xgboost_tuned = pickle.load(f)

In [4]:
with open('svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)

In [5]:
with open('best_knn_model.pkl', 'rb') as f:
    knn_model = pickle.load(f)

C:\Users\Nath\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
from sklearn.ensemble import VotingClassifier

In [7]:
classifiers = [('Logistic Regression', logistic_regression_model),
               ('XGBoost', xgboost_tuned),
               ('SVM', svm_model),
               ('KNN', knn_model)]

voting_classifier = VotingClassifier(estimators=classifiers, voting='hard')

In [8]:
df = pd.read_csv('./Complete-data.csv')

X = df.drop(columns=['Landslide'])
y = df['Landslide']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
voting_classifier.fit(X_train, y_train)
predictions = voting_classifier.predict(X_test)

In [10]:
df.columns

Index(['Landslide', 'Aspect', 'Curvature', 'Earthquake', 'Elevation', 'Flow',
       'Lithology', 'NDVI', 'NDWI', 'Plan', 'Precipitation', 'Profile',
       'Slope'],
      dtype='object')

In [11]:
models = [logistic_regression_model, xgboost_tuned, svm_model, knn_model, voting_classifier]
model_names = ['Logistic Regression', 'XGBoost', 'SVM', 'KNN', 'Voting Classifier']

In [12]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

attribute_columns = ['Aspect', 'Curvature', 'Earthquake', 'Elevation', 'Flow', 'Lithology', 
                     'NDVI', 'NDWI', 'Plan', 'Precipitation', 'Profile', 'Slope']

sliders = {col: widgets.IntSlider(min=1, max=5, description=col) for col in attribute_columns}

model_selection = widgets.Dropdown(options=model_names, description='Model')

predict_button = widgets.Button(description='Predict')

output = widgets.Output()

In [13]:
def make_prediction(b):
    with output:
        output.clear_output()
        selected_model = models[model_names.index(model_selection.value)]
        input_data = {col: [slider.value] for col, slider in sliders.items()}
        input_df = pd.DataFrame(input_data)
        prediction = selected_model.predict(input_df)
        print(f"Predicted landslide: {prediction}")

In [14]:
predict_button.on_click(make_prediction)

In [15]:
display(widgets.VBox(list(sliders.values()) + [model_selection, predict_button, output]))

In [16]:
with open('voting_classifier_model.pkl', 'wb') as f:
    pickle.dump(voting_classifier, f)